In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context="notebook", style="darkgrid", palette="deep", font="sans-serif", font_scale=1, color_codes=True)

# Data :

In [2]:
data= pd.read_csv("train.csv")

In [3]:
data["Date"].max() ## 2020-03-20


'2020-03-24'

In [4]:
data["Date"] = data["Date"].apply(lambda x: x.replace("-",""))
data["Date"]  = data["Date"].astype(int)

In [5]:
data = data.drop(['Province/State'],axis=1)
data = data.dropna()
data.isnull().sum()


Id                0
Country/Region    0
Lat               0
Long              0
Date              0
ConfirmedCases    0
Fatalities        0
dtype: int64

In [6]:
data.head()

,Id,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,Afghanistan,33.0,65.0,20200122,0.0,0.0
1,2,Afghanistan,33.0,65.0,20200123,0.0,0.0
2,3,Afghanistan,33.0,65.0,20200124,0.0,0.0
3,4,Afghanistan,33.0,65.0,20200125,0.0,0.0
4,5,Afghanistan,33.0,65.0,20200126,0.0,0.0


In [7]:
test = pd.read_csv("test.csv")   
test["Date"] = test["Date"].apply(lambda x: x.replace("-",""))
test["Date"]  = test["Date"].astype(int)

test["Lat"]  = test["Lat"].fillna(12.5211)
test["Long"]  = test["Long"].fillna(69.9683)
test.isnull().sum()


ForecastId           0
Province/State    6622
Country/Region       0
Lat                  0
Long                 0
Date                 0
dtype: int64

In [8]:
test["Date"].min() ## 2020-03-12

20200312

In [9]:
x = data[['Lat', 'Long', 'Date']]
y1 = data[['ConfirmedCases']]
y2 = data[['Fatalities']]
x_test = test[['Lat', 'Long', 'Date']]


In [10]:
from sklearn.tree import DecisionTreeClassifier
Tree_model = DecisionTreeClassifier(criterion='entropy')

<a id='Model'></a>

# Model : 

In [11]:
##
Tree_model.fit(x,y1)
pred1 = Tree_model.predict(x_test)
pred1 = pd.DataFrame(pred1)
pred1.columns = ["ConfirmedCases_prediction"]

In [12]:
pred1.head()

,ConfirmedCases_prediction
0,7.0
1,7.0
2,11.0
3,16.0
4,16.0


In [13]:
Tree_model.fit(x,y2)
pred2 = Tree_model.predict(x_test)
pred2 = pd.DataFrame(pred2)
pred2.columns = ["Death_prediction"]

In [14]:
pred2.head()

,Death_prediction
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [15]:
Sub = pd.read_csv("submission.csv")
Sub.columns
sub_new = Sub[["ForecastId"]]

In [16]:
OP = pd.concat([pred1,pred2,sub_new],axis=1)
OP.head()
OP.columns = ['ConfirmedCases', 'Fatalities', 'ForecastId']
OP = OP[['ForecastId','ConfirmedCases', 'Fatalities']]


In [17]:
OP["ConfirmedCases"] = OP["ConfirmedCases"].astype(int)
OP["Fatalities"] = OP["Fatalities"].astype(int)


In [18]:
OP.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,7,0
1,2,7,0
2,3,11,0
3,4,16,0
4,5,16,0


In [19]:

#### By using DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor(random_state = 0) 

In [20]:
regressor.fit(x,y1)
pred_r1 = regressor.predict(x_test)
pred_r1 = pd.DataFrame(pred_r1)
pred_r1.columns = ["ConfirmedCases_prediction"]


In [21]:
regressor.fit(x,y2)
pred_r2 = regressor.predict(x_test)
pred_r2 = pd.DataFrame(pred_r2)
pred_r2.columns = ["Death_prediction"]

In [22]:
OP_dr = pd.concat([sub_new,pred_r1,pred_r2],axis=1)
OP_dr.head()
OP_dr.columns = [ 'ForecastId','ConfirmedCases', 'Fatalities']


In [23]:
OP_dr["ConfirmedCases"] = OP_dr["ConfirmedCases"].astype(int)
OP_dr["Fatalities"] = OP_dr["Fatalities"].astype(int)
OP_dr.head()


,ForecastId,ConfirmedCases,Fatalities
0,1,7,0
1,2,7,0
2,3,11,0
3,4,19,0
4,5,19,0


In [24]:
## Random Forest regressor
from sklearn.ensemble import RandomForestRegressor
rand_reg = RandomForestRegressor()


In [25]:
rand_reg.fit(x,y1)
pred_ra1 = rand_reg.predict(x_test)
pred_ra1 = pd.DataFrame(pred_ra1)
pred_ra1.columns = ["ConfirmedCases_prediction"]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [26]:
rand_reg.fit(x,y2)
pred_ra2 = rand_reg.predict(x_test)
pred_ra2 = pd.DataFrame(pred_ra2)
pred_ra2.columns = ["Death_prediction"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [27]:
OP_ra = pd.concat([sub_new,pred_ra1,pred_ra2],axis=1)
OP_ra.head()
OP_ra.columns = [ 'ForecastId','ConfirmedCases', 'Fatalities']


In [28]:
OP_ra["ConfirmedCases"] = OP_ra["ConfirmedCases"].astype(int)
OP_ra["Fatalities"] = OP_ra["Fatalities"].astype(int)
OP_ra.head()


,ForecastId,ConfirmedCases,Fatalities
0,1,8,0
1,2,8,0
2,3,11,0
3,4,19,0
4,5,19,0


<a id='Out_sub'></a>

# Output submission :

In [29]:
OP_ra.to_csv("new_submission.csv",index=False)